<a href="https://colab.research.google.com/github/rahiakela/building-computer-vision-applications-using-artificial-neural-networks/blob/master/6-deep-learning-in-object-detection/1_training_object_detection_model_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Object Detection Model Using TensorFlow

Object detection models are very compute-intensive and require both a lot of memory and a powerful processor. Most general-purpose laptops or computers may not be able to handle the computations necessary to build and train an object detection model.

Thankfully, Google provides a limited amount of GPU-based computing
for free. It has been proven that these models run many folds faster on a GPU than on a CPU. Therefore, it is important to learn how to train a model on a GPU.

Let’s first define what our learning objective is and how we want to achieve it.

- **Objective**: Learn how to train an object detection model using Keras
and TensorFlow.
- **Dataset**: The Oxford-IIIT Pet dataset, which is freely available at
robots.ox.ac.uk/~vgg/data/pets/. The dataset consists of 37
categories of pets with roughly 200 images for each class. The images
have large variations in scale, pose, and lighting. They are already
annotated with bounding boxes and labeled.

- **Important note**: At the time of writing this book, TensorFlow version 2
does not support the training of custom models for object detection.
Therefore, we will use TensorFlow version 1.15 to train the model.
The TensorFlow team and the open source community are working to migrate the version 1 code to support the training of custom object
detection models in version 2.

## Setup

Google Colab is an interactive programming environment and does not give direct
access to the underlying operating system. You can invoke the shell using `%%shell`, which remains active within a single block of code cells it is invoked from. You can invoke the shell from as many code blocks as needed.

To set up our environment, we will follow the following steps:

1. Install the necessary libraries needed to execute our TensorFlow
code and train our model.

In [ ]:
!pip install tensorflow==1.15

In [ ]:
# Installing the Necessary Libraries and Packages
%%shell
%tensorflow_version 1.x
sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
pip install --user Cython
pip install --user contextlib2
pip install --user pillow
pip install --user lxml
pip install --user matplotlib

2. Download the TensorFlow “models” project from the GitHub repository, and build and install it in your working environment.

In [ ]:
# Downloading the TensorFlow Models Project, Building It, and Setting It Up
%%shell
mkdir computer_vision
cd computer_vision
git clone https://github.com/ansarisam/models.git
# Official repository
# git clone https://github.com/tensorflow/models.git

# The research directory contains a large number of models that are created and maintained by researchers and not officially supported yet.
cd models/research

# builds the object detection–related source code using the Protobuf compiler.
protoc object_detection/protos/*.proto --python_out=.

# set the PYTHONPATH environment variable to the research and research/slim directories.
export PYTHONPATH=$PYTHONPATH:/content/computer_vision/models/research
export PYTHONPATH=$PYTHONPATH:/content/computer_vision/models/research/slim

# executes a build command using setup.py
python setup.py build
# installs the object detection models in our working environment.
python setup.py install

## Downloading the Oxford-IIIT Pet Dataset

We will download the annotated and labeled pet dataset from the official website to a directory in our Colab workspace.

In [ ]:
# Downloading and Uncompressing the Images and Annotations of the Pet Dataset
%%shell
cd computer_vision
mkdir petdata
cd petdata
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
tar -xvf annotations.tar.gz
tar -xvf images.tar.gz

## Generating TensorFlow TFRecord Files

TFRecord is a simple format for storing a sequence of binary records. The data in TFRecord is serialized and stored in smaller chunks (e.g., 100MB to 200MB), which makes them more efficient to transfer across networks and read serially. You will learn more about TFRecord, its format, and how to convert images and associated annotations in the TFRecord file format.

Object detection algorithms take TFRecord files as input to the neural network.
TensorFlow provides a Python script to convert the Oxford pet image annotation files to a set of TFRecord files.

In [ ]:
# Converting Image Annotation Files to TFRecord Files
%%shell
cd computer_vision/models/research

python object_detection/dataset_tools/create_pet_tf_record.py \
       --label_map_path=object_detection/data/pet_label_map.pbtxt \
       --data_dir=/content/computer_vision/petdata \
       --output_dir=/content/computer_vision/petdata/

The Python script, create_pet_tf_record.py, that takes the following parameters:

- **label_map_path**: This file has the mapping of an ID (starting from 1)
and corresponding class name. For the pet dataset, the mapping file
is already available in the object_detection/data/pet_label_map.
pbtxt file.
- **data_dir**: This is the parent directory of the images and annotations
subdirectories.
- **output_dir**: This is the destination directory where the TFRecord
files will be stored. You can give any existing directory name. After
conversion of images and annotations, the TFRecord files will be
saved in this directory.

After this executes, it creates a set of *.record files in the output_directory. The script, create_pet_tf_record.py, creates both training and evaluation sets.

## Downloading a Pre-trained Model for Transfer Learning

Training a state-of-the-art object detection model from scratch takes several days, even with GPUs. To speed up the training, we will download an existing model trained on a different dataset, such as COCO, and reuse some of its parameters, including the weights, to initialize our new model. Reusing the weights and parameters from a pre-trained model to train a new model is called transfer learning.

A collection of object detection models, trained on COCO and other datasets, is
located at “TensorFlow detection model zoo” (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md).

For our training, we will download the ssd_inception_v2_coco model from
http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz. You can download any of the trained models and follow the rest of the steps to train your own model.

In [24]:
# Downloading a Pre-trained SSD Inception Object Detection Model
%%shell
cd computer_vision
mkdir pre-trained-model
cd pre-trained-model
wget http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz
tar -xvf ssd_inception_v2_coco_2018_01_28.tar.gz

--2020-11-17 10:17:29--  http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.203.128, 2404:6800:4008:c04::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.203.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278114232 (265M) [application/x-tar]
Saving to: ‘ssd_inception_v2_coco_2018_01_28.tar.gz’

ssd_inception_v2_co 100%[===================>] 265.23M  32.4MB/s    in 8.2s    

2020-11-17 10:17:39 (32.4 MB/s) - ‘ssd_inception_v2_coco_2018_01_28.tar.gz’ saved [278114232/278114232]

ssd_inception_v2_coco_2018_01_28/
ssd_inception_v2_coco_2018_01_28/model.ckpt.index
ssd_inception_v2_coco_2018_01_28/checkpoint
ssd_inception_v2_coco_2018_01_28/pipeline.config
ssd_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
ssd_inception_v2_coco_2018_01_28/model.ckpt.meta
ssd_inception_v2_coco_2018_01_28/saved_model/
ssd_inception

## Configuring the Object Detection Pipeline